In [228]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Auxiliar Methods

In [229]:
def showImage(imageTitle, image):
    cv2.imshow(imageTitle, image)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def showImageArray(tituloArray, arrayImagens):
    for index in range(len(arrayImagens)):
        showImage(tituloArray + ": " + str(index), arrayImagens[index])

## Caracteristicas Imagens

In [230]:
numberOfImages = 9
width = 768
height = 1024

## Carregar Imagens Originais

In [231]:
imagesOriginal = [None] * numberOfImages
initialIndex = 1000697
currentIndex = initialIndex

for index in range(numberOfImages):
    while imagesOriginal[index] is None:
        imagesOriginal[index] = cv2.imread("P" + str(currentIndex) + "s.jpg")
        currentIndex = currentIndex + 1
        
# showImageArray("Imagem Original", imagesOriginal)

## Obter Imagens Grayscale

In [232]:
imagesGrayscale = [None] * numberOfImages

for index in range(numberOfImages):
    imagesGrayscale[index] = cv2.cvtColor(imagesOriginal[index], cv2.COLOR_BGR2GRAY)
    
# showImageArray("Imagem Grayscale", imagesGrayscale)

## Obter Thresholds e Imagens Binárias

In [233]:
imagesBinary = [None] * numberOfImages
imagesThresholds = [None] * numberOfImages

for index in range(numberOfImages):
    imagesThresholds[index], imagesBinary[index] = cv2.threshold(imagesGrayscale[index], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# showImageArray("Imagem Binaria", imagesBinary)

# TESTES

In [234]:
imagemBinaria = imagesBinary[0]

#Melhorar a imagem
mascara5x5 = np.ones((5, 5), np.uint8)
mascara5x5[0, 0] = 0
mascara5x5[0, 4] = 0
mascara5x5[4, 0] = 0
mascara5x5[4, 4] = 0

mascara7x7 = np.ones((7, 7), np.uint8)
mascara7x7[0, 0] = 0
mascara7x7[0, 6] = 0
mascara7x7[6, 0] = 0
mascara7x7[6, 6] = 0

mascara9x9 = np.ones((9, 9), np.uint8)
mascara9x9[0, 0] = 0
mascara9x9[0, 8] = 0
mascara9x9[8, 0] = 0
mascara9x9[8, 8] = 0

#Erosão
erosao1 = cv2.erode(imagemBinaria, mascara5x5, iterations = 1)
erosao2 = cv2.erode(imagemBinaria, mascara5x5, iterations = 10)
erosao3 = cv2.erode(imagemBinaria, mascara5x5, iterations = 30)
erosao4 = cv2.erode(imagemBinaria, mascara5x5, iterations = 50)

erosao = [erosao1, erosao2, erosao3, erosao4]
 
#Fecho
fechoV1 = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mascara5x5)
fechoV2 = cv2.morphologyEx(fechoV1, cv2.MORPH_CLOSE, mascara5x5)
fechoV3 = cv2.morphologyEx(fechoV2, cv2.MORPH_CLOSE, mascara5x5)

fecho = [fechoV1, fechoV2, fechoV3] #Várias iterações não tem efeito

fecho5X5 = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mascara5x5)
fecho7X7 = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mascara7x7)
fecho9X9 = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mascara9x9)

##Visualizar as Imagens
showImage("Binária", imagemBinaria)

#showImageArray("Erosao", erosao)
#showImageArray("Fecho 5x5", fecho)
showImage("Fecho 5X5", fecho5X5)
showImage("Fecho 7X7", fecho7X7)
showImage("Fecho 9X9", fecho9X9)

imagemBinariaColada = imagesBinary[4]
showImage("Bin Problema", imagemBinariaColada)


fecho5X5 = cv2.morphologyEx(imagemBinariaColada, cv2.MORPH_CLOSE, mascara5x5)
fecho7X7 = cv2.morphologyEx(imagemBinariaColada, cv2.MORPH_CLOSE, mascara7x7)
fecho9X9 = cv2.morphologyEx(imagemBinariaColada, cv2.MORPH_CLOSE, mascara9x9)

showImage("Fecho5X5", fecho5X5)
showImage("Fecho7X7", fecho7X7)
showImage("Fecho9X9", fecho9X9)

#O engenheiro disse-me para usar a dilatação também com os discos.

# V2

Ideia básica:<br>
Agarramos na primeira imagem onde temos todas as moedas bem dispersas.<br>
Corrigimos as moedas com as máscaras.<br>
Separamo-las.<br>
Fazemos máscaras para cada uma.<br>
A partir disto, percorremos as imagens todas e procuramos correspondências às mesmas.<br>

Não esquecer de caracterizar os valores das moedas.
Atribuir gamas de valores à característica.

In [235]:
mask = np.ones((9, 9), np.uint8)
mask[0, 0] = 0
mask[0, 8] = 0
mask[8, 0] = 0
mask[8, 8] = 0

#Imagem após a Operação Morfológica
filledImage = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mask)
erodedImage = cv2.morphologyEx(filledImage, cv2.MORPH_OPEN, mask)

ret, thresh = cv2.threshold(erodedImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 

#imagesThresholds = np.asarray(imagesThresholds, dtype=np.uint8)

#Próximo passo, separar objectos
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image = cv2.drawContours(erodedImage, contours, -1, (255, 255, 255), 3)
#connectedComponents

# showImage("bin", imagesBinary[0])
# showImage("Filled", erodedImage)
# showImage("Contour Filled", image)

#num_labels, labels_im = cv2.connectedComponents(image)
#labels_im = np.asarray(labels_im, dtype=np.uint8)
#labels_im[labels_im != 0] = 255

numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, 8, cv2.CV_32S)

#Pelo que percebi, o numLabel = 0 é o fundo preto.
#A partir dai, já são os objectos

# print(numLabels)
# print()
# print(labels)
# print()
# print(stats)

initialPointX = stats[1, cv2.CC_STAT_LEFT]
initialPointY = stats[1, cv2.CC_STAT_TOP]
width = stats[1, cv2.CC_STAT_WIDTH]
height = stats[1, cv2.CC_STAT_HEIGHT]
area = stats[1, cv2.CC_STAT_AREA]
(centroidX, centroidY) = centroids[1]

#rect = cv2.rectangle(output, (initialPointX, initialPointY), (initialPointX + width, initialPointY + height), (255, 255, 255), 1)
#circle = cv2.circle(output, (int(centroidX), int(centroidY)), 1, (255, 0, 255), -1)

componentMask = (labels == 1).astype(np.uint8) * 255
print(initialPointX)
print(initialPointY)
print(width)
print(height)

#Não consigo cortar a imagem como era suposto
#componentMask = erodedImage[initialPointX : initialPointX + width, initialPointY : initialPointY + height]

print(componentMask)

#showImage("Bin", imagemBinaria)
#showImage("Filled", filledImage)
#showImage("Filled", labels_im)
cv2.imshow("Eroded", erodedImage)
cv2.imshow("Mask", componentMask)
cv2.waitKey(0)
cv2.destroyAllWindows()

510
90
148
149
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Para o Engenheiro

In [236]:
import numpy as np
import cv2

def showImage(imageTitle, image):
    cv2.imshow(imageTitle, image)
    cv2.waitKey()
    cv2.destroyAllWindows()

#Carregar Imagem Original, Converter para GrayScale e depois para Binaria
imagemOriginal = cv2.imread("P1000697s.jpg")
imagemGrayscale = cv2.cvtColor(imagemOriginal, cv2.COLOR_BGR2GRAY)
threshold, imagemBinaria = cv2.threshold(imagemGrayscale, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#Elemento Estruturante Disco(9)
mask = np.ones((9, 9), np.uint8)
mask[0, 0] = 0
mask[0, 8] = 0
mask[8, 0] = 0
mask[8, 8] = 0

#Imagem após a Operação Morfológica
filledImage = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mask)
erodedImage = cv2.morphologyEx(filledImage, cv2.MORPH_OPEN, mask)

#Preciso de atualizar o threshold para o findContours e connectedComponentsWithStats
ret, thresh = cv2.threshold(erodedImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 

#Próximo passo, separar objectos
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image = cv2.drawContours(erodedImage, contours, -1, (255, 255, 255), 3)

#
numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, 8, cv2.CV_32S)

#labels[0] é o fundo (tudo preto)
initialPointX = stats[1, cv2.CC_STAT_LEFT]
initialPointY = stats[1, cv2.CC_STAT_TOP]
width = stats[1, cv2.CC_STAT_WIDTH]
height = stats[1, cv2.CC_STAT_HEIGHT]
area = stats[1, cv2.CC_STAT_AREA]
(centroidX, centroidY) = centroids[1]

componentMask = (labels == 1).astype(np.uint8) * 255
#print(initialPointX)
#print(initialPointY)
#print(width)
#print(height)

#IMPORTANTE!
#O corte tem de ser y's e depois x's e não o convencional [x, y]
componentMaskCut = erodedImage[initialPointY : initialPointY + height, initialPointX : initialPointX + width]

print(area)
raio = width #Isto é o diametro, teoricamente
print(width)
print(raio)


#getStructuralElement
disco = cv2.MORPH_ELLIPSE
size = (int(raio), int(raio))
elementoEstruturante = cv2.getStructuringElement(disco, size)
elementoEstruturante = elementoEstruturante * 255

print("CMK", componentMaskCut.shape)
print("EE", elementoEstruturante.shape)

#Teste da operação AND

rectangle = np.zeros((148, 148), dtype="uint8")
cv2.rectangle(rectangle, (24, 24), (124, 124), 255, -1)
#cv2.imshow("Rectangle", rectangle)


result = cv2.bitwise_and(componentMaskCut[1:,:], elementoEstruturante)
#cv2.imshow("Coin", result)
#cv2.waitKey(0)
result = cv2.bitwise_and(rectangle, elementoEstruturante)
#cv2.imshow("Square", result)

#--------------------
# Perform template matching
result = cv2.matchTemplate(erodedImage, elementoEstruturante, cv2.TM_CCOEFF_NORMED)

# Define a threshold to determine confidence in matches
threshold = 0.9

# Find the locations where the result is above the threshold
locations = np.where(result >= threshold)

print(locations)


#cv2.imshow("", componentMaskCut)
#cv2.imshow("Elemento Estruturante", elementoEstruturante)
#a = erodedImage[initialPointX - 100 : initialPointX + 100, initialPointY - 100 : initialPointY + 100]
#print(a)
#print(erodedImage)
#print("Sum Eroded: ", np.sum(erodedImage==255))
#b = componentMask[initialPointX : initialPointX + width, initialPointY : initialPointY + height]
#print(b == 255)
#print("Sum: ", np.sum(componentMask==255))

#cv2.imshow("Mascara Moeda Isolada", erodedImage)
#cv2.imshow("Mascara Moeda Isolada", componentMask)
#cv2.imshow("Mascara Moeda Isolada", componentMaskCut)
#cv2.imshow("Teste", teste)
cv2.waitKey(0)
cv2.destroyAllWindows()

17358
148
148
CMK (149, 148)
EE (148, 148)
(array([87, 87, 88, 88, 88, 88, 89, 89, 89, 89, 90, 90, 90, 91, 91, 91, 92],
      dtype=int64), array([509, 510, 508, 509, 510, 511, 508, 509, 510, 511, 508, 509, 510,
       508, 509, 510, 509], dtype=int64))


#### Características da Primeira Imagem

In [257]:
valorMoedas = [0.05, 0.01, 0.50, 0.02, 0.10, 0.20, 1]
diametroMascaraMoedas = np.zeros(len(valorMoedas), np.uint8)

for componenteIndex in range(numLabels):
    if componenteIndex != 0 and componenteIndex != 7:
        
        #Características dos componentes
        initialPointX = stats[componenteIndex, cv2.CC_STAT_LEFT]
        initialPointY = stats[componenteIndex, cv2.CC_STAT_TOP]
        width = stats[componenteIndex, cv2.CC_STAT_WIDTH]
        height = stats[componenteIndex, cv2.CC_STAT_HEIGHT]
        area = stats[componenteIndex, cv2.CC_STAT_AREA]
        (centroidX, centroidY) = centroids[componenteIndex]
        
        #Isolar o componente
        componentMaskCut = erodedImage[initialPointY : initialPointY + height, initialPointX : initialPointX + width]
        
        #Obter o Elemento Estruturante do Tamanho do Componente
        disco = cv2.MORPH_ELLIPSE
        size = (int(width), int(width))
        elementoEstruturante = cv2.getStructuringElement(disco, size) * 255
        
        #Adicionar o tamanho da Moeda ao array
        if componenteIndex == 8:
            diametroMascaraMoedas[componenteIndex - 2] = int(width)
        else:
            diametroMascaraMoedas[componenteIndex - 1] = int(width)

#### Observação de Resultados

In [258]:
print("Valor das Moedas: ", valorMoedas)
print("Diametro das Moedas: ", diametroMascaraMoedas)

Valor das Moedas:  [0.05, 0.01, 0.5, 0.02, 0.1, 0.2, 1]
Diametro das Moedas:  [148 113 170 134 139 158 163]


#### Teste para a primeira Imagem

In [280]:
imagemOriginal = cv2.imread("P1000697s.jpg")
imagemGrayscale = cv2.cvtColor(imagemOriginal, cv2.COLOR_BGR2GRAY)
threshold, imagemBinaria = cv2.threshold(imagemGrayscale, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#Elemento Estruturante Disco(9)
mask = np.ones((9, 9), np.uint8)
mask[0, 0] = 0
mask[0, 8] = 0
mask[8, 0] = 0
mask[8, 8] = 0

#Imagem após a Operação Morfológica
filledImage = cv2.morphologyEx(imagemBinaria, cv2.MORPH_CLOSE, mask)
erodedImage = cv2.morphologyEx(filledImage, cv2.MORPH_OPEN, mask)

#Preciso de atualizar o threshold para o findContours e connectedComponentsWithStats
ret, thresh = cv2.threshold(erodedImage, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 

#Próximo passo, separar objectos
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image = cv2.drawContours(erodedImage, contours, -1, (255, 255, 255), 3)

#
numLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, 8, cv2.CV_32S)

valorRealNaImagem = 2.08
valorNaImagemObtido = 0

for componenteIndex in range(numLabels):
    if componenteIndex != 0:
        
        #Características dos componentes
        initialPointX = stats[componenteIndex, cv2.CC_STAT_LEFT]
        initialPointY = stats[componenteIndex, cv2.CC_STAT_TOP]
        width = stats[componenteIndex, cv2.CC_STAT_WIDTH]
        height = stats[componenteIndex, cv2.CC_STAT_HEIGHT]
        area = stats[componenteIndex, cv2.CC_STAT_AREA]
        (centroidX, centroidY) = centroids[componenteIndex]
        
        #Isolar o componente
        componentMaskCut = erodedImage[initialPointY : initialPointY + height, initialPointX : initialPointX + width]
        
        #Definir um limiar de aceitação do padrão 
        threshold = 0.9
        
        #Guardar as posições previamente descobertas
        pointXBefore = -1
        pointYBefore = -1
        
        #Valores dos componentes por ordem
        ordem = [0.05, 0.01, 0.50, 0.02, 0.10, 0.20, 0.20, 1]
        
        print("Avaliação do valor ", ordem[componenteIndex - 1])
        
        #Avaliação do Componente
        for mascaraIndex in range(len(valorMoedas)):
            disco = cv2.MORPH_ELLIPSE
            size = (diametroMascaraMoedas[mascaraIndex], diametroMascaraMoedas[mascaraIndex])
            elementoEstruturante = cv2.getStructuringElement(disco, size) * 255
        
            #Procura padrão nas Imagem
            result = cv2.matchTemplate(erodedImage, elementoEstruturante, cv2.TM_CCOEFF_NORMED)

            #Encontrar as localizações.
            locations = np.where(result >= threshold)
        
            #Comparação de cada máscara das moedas preparadas com a moeda real.
            print("A comparar com máscara do valor ", valorMoedas[mascaraIndex])
            print("Localizações encontradas: ", locations)
            print()
        
            for locationIndex in range(len(locations[0])):
                
                if(locations[0][locationIndex] - pointXBefore > width or locations[1][locationIndex] - pointYBefore > height):
                    
                    if(diametroMascaraMoedas[mascaraIndex] >= width - 5 and diametroMascaraMoedas[mascaraIndex] <= width + 5):
                        valorNaImagemObtido = valorNaImagemObtido + valorMoedas[mascaraIndex]
                
                        pointXBefore = locations[0][locationIndex]
                        pointYBefore = locations[1][locationIndex]
            
        print("Fim da avaliação do valor ", ordem[componenteIndex - 1])
        print()

print("Valor Real Na Imagem: ", valorRealNaImagem)
print("Valor Obtido Na Imagem: ", valorNaImagemObtido)

Avaliação do valor  0.05
A comparar com máscara do valor  0.05
Localizações encontradas:  (array([87, 87, 88, 88, 88, 88, 89, 89, 89, 89, 90, 90, 90, 91, 91, 91, 92],
      dtype=int64), array([509, 510, 508, 509, 510, 511, 508, 509, 510, 511, 508, 509, 510,
       508, 509, 510, 509], dtype=int64))

A comparar com máscara do valor  0.01
Localizações encontradas:  (array([], dtype=int64), array([], dtype=int64))

A comparar com máscara do valor  0.5
Localizações encontradas:  (array([164, 164, 164, 165, 165, 165, 165, 166, 166, 166, 166, 166, 167,
       167, 167, 167, 167, 168, 168, 168, 168, 168, 169, 169, 169, 420,
       420, 420, 421, 421, 421, 421, 421, 422, 422, 422, 422, 422, 422,
       423, 423, 423, 423, 423, 424, 424, 424], dtype=int64), array([698, 699, 700, 697, 698, 699, 700, 697, 698, 699, 700, 701, 697,
       698, 699, 700, 701, 697, 698, 699, 700, 701, 698, 699, 700, 573,
       574, 575, 571, 572, 573, 574, 575, 571, 572, 573, 574, 575, 576,
       571, 572, 573, 57

A comparar com máscara do valor  0.05
Localizações encontradas:  (array([87, 87, 88, 88, 88, 88, 89, 89, 89, 89, 90, 90, 90, 91, 91, 91, 92],
      dtype=int64), array([509, 510, 508, 509, 510, 511, 508, 509, 510, 511, 508, 509, 510,
       508, 509, 510, 509], dtype=int64))

A comparar com máscara do valor  0.01
Localizações encontradas:  (array([], dtype=int64), array([], dtype=int64))

A comparar com máscara do valor  0.5
Localizações encontradas:  (array([164, 164, 164, 165, 165, 165, 165, 166, 166, 166, 166, 166, 167,
       167, 167, 167, 167, 168, 168, 168, 168, 168, 169, 169, 169, 420,
       420, 420, 421, 421, 421, 421, 421, 422, 422, 422, 422, 422, 422,
       423, 423, 423, 423, 423, 424, 424, 424], dtype=int64), array([698, 699, 700, 697, 698, 699, 700, 697, 698, 699, 700, 701, 697,
       698, 699, 700, 701, 697, 698, 699, 700, 701, 698, 699, 700, 573,
       574, 575, 571, 572, 573, 574, 575, 571, 572, 573, 574, 575, 576,
       571, 572, 573, 574, 575, 572, 573, 574], d

A comparar com máscara do valor  0.02
Localizações encontradas:  (array([205, 205, 205, 205, 206, 206, 206, 206, 207, 207, 207, 207, 208,
       208, 208, 208], dtype=int64), array([51, 52, 53, 54, 51, 52, 53, 54, 51, 52, 53, 54, 51, 52, 53, 54],
      dtype=int64))

A comparar com máscara do valor  0.1
Localizações encontradas:  (array([203, 203, 203, 203, 204, 204, 204, 204, 204, 205, 205, 205, 205,
       205, 205, 206, 206, 206, 206, 207, 207], dtype=int64), array([49, 50, 51, 52, 49, 50, 51, 52, 53, 48, 49, 50, 51, 52, 53, 49, 50,
       51, 52, 50, 51], dtype=int64))

A comparar com máscara do valor  0.2
Localizações encontradas:  (array([330, 330, 330, 331, 331, 331, 331, 331, 332, 332, 332, 332, 332,
       332, 333, 333, 333, 333, 333, 333, 334, 334, 334, 334, 334, 335,
       335, 335, 335, 376, 376, 376, 376, 377, 377, 377, 377, 377, 377,
       378, 378, 378, 378, 378, 378, 379, 379, 379, 379, 379, 379, 380,
       380, 380, 380, 381, 381, 381], dtype=int64), array([193, 19

## Melhorar as Imagens

In [239]:
imagesClosedHoles = [None] * numberOfImages

for index in range(numberOfImages):
    imagesClosedHoles[index] = cv2.morphologyEx(imagesBinary[index], cv2.MORPH_CLOSE, kernel=np.ones((5,5),np.uint8))
    
# showImageArray("Imagem Buracos Fechados", imagesClosedHoles)

### Tenho de fazer a divisão dos objectos primeiro.

In [240]:
imagemTeste = np.zeros([10, 10])
#Figura 1
imagemTeste[0, 1] = 1
imagemTeste[1, 1] = 1
imagemTeste[1, 2] = 1
imagemTeste[1, 3] = 1
imagemTeste[2, 2] = 1
imagemTeste[2, 3] = 1
imagemTeste[2, 4] = 1
imagemTeste[2, 5] = 1
imagemTeste[3, 2] = 1
imagemTeste[3, 3] = 1
imagemTeste[4, 2] = 1
imagemTeste[4, 3] = 1
imagemTeste[4, 4] = 1
#Figura 2
imagemTeste[4, 9] = 1
imagemTeste[5, 8] = 1
imagemTeste[5, 9] = 1
imagemTeste[6, 8] = 1
imagemTeste[6, 9] = 1

#Figura 3
imagemTeste[8, 5] = 1
imagemTeste[9, 3] = 1
imagemTeste[9, 4] = 1
imagemTeste[9, 5] = 1

imagemAuxiliar = imagemTeste * 0
currentLabel = 1
labels = {}

#Não esquecer de adicionar um dicionário para associar as diferentes labels.

for row in range(imagemTeste.shape[0]):
    for col in range(imagemTeste.shape[1]):
        if imagemTeste[row, col] != 0:            
    
            hasLabelPreviousRow = False
            hasLabelPreviousCol = False
            hasLabelPreviousDiagonalLeft = False
            hasLabelPreviousDiagonalRight = False
    
            #comparar diagonal esquerda
            if row > 0 and col > 0:
                if imagemAuxiliar[row  - 1, col - 1] != 0:
                    hasLabelPreviousDiagonalLeft = True
            
            #comparar diagonal direita
            if row > 0 and col < imagemAuxiliar.shape[1] - 1:
                if imagemAuxiliar[row  - 1, col + 1] != 0:
                    hasLabelPreviousDiagonalRight = True    
                
            #comparar colunas
            if col > 0:
                if imagemAuxiliar[row, col - 1] != 0:
                    hasLabelPreviousCol = True
            #comparar linhas
            if row > 0:
                if imagemAuxiliar[row  - 1, col] != 0:
                    hasLabelPreviousRow = True
            
            #Associar Labels
            #Inicio - diagonal direita - Label        
            if hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == True and hasLabelPreviousRow == True:
                label = imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row, col - 1], imagemAuxiliar[row - 1, col]])
                
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == True and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == False and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row - 1, col]])
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == False and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row - 1, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == True and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row, col - 1], imagemAuxiliar[row - 1, col]])
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == True and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == False and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1], imagemAuxiliar[row - 1, col]]) # Parece dar erro
            
            elif hasLabelPreviousDiagonalRight == True and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == False and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col + 1]])
            #Fim - diagonal direita - Label
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft and hasLabelPreviousCol == True and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row, col - 1], imagemAuxiliar[row - 1, col]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == True and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == False and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col - 1], imagemAuxiliar[row - 1, col]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == True and hasLabelPreviousCol == False and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == True and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row, col - 1], imagemAuxiliar[row - 1, col]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == True and hasLabelPreviousRow == False:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row, col - 1]])
            
            elif hasLabelPreviousDiagonalRight == False and hasLabelPreviousDiagonalLeft == False and hasLabelPreviousCol == False and hasLabelPreviousRow == True:
                imagemAuxiliar[row, col] = np.min([imagemAuxiliar[row - 1, col]])
                                               
            else:
                imagemAuxiliar[row, col] = currentLabel
                currentLabel = currentLabel + 1
                    
# print(imagemTeste)
print()
print(imagemAuxiliar)


[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 2.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 3. 0. 0. 0. 0.]
 [0. 0. 0. 4. 3. 3. 0. 0. 0. 0.]]
